# Dataset

1. Dataset used : Synthetic Financial Datasets For Fraud Detection [ https://www.kaggle.com/ntnu-testimon/paysim1 ]
2. Description : Paysim synthetic dataset of mobile money transactions. Each step represents an hour of simulation. This dataset is scaled down 1/4 of the original dataset which is presented in the paper "PaySim: A financial mobile money simulator for fraud detection".

# Initializing Spark

In [43]:
# Import findspark to read SPARK_HOME and HADOOP_HOME
import findspark
findspark.init()
# Import required library
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.appName("Simple data mining with Synthetic Financial Dataset").getOrCreate()
    

In [44]:
# Print Spark object ID
print(spark)

# Loading Dataset 

In [45]:
df = spark.read.csv("E:/BD/PS_20174392719_1491204439457_log.csv", header=True, inferSchema=True)

In [46]:
#Show Dataset

df.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

In [47]:
#Count how many rows in dataset

df.count()

6362620

In [48]:
#Show Dataset Schema

df.schema

StructType(List(StructField(step,IntegerType,true),StructField(type,StringType,true),StructField(amount,DoubleType,true),StructField(nameOrig,StringType,true),StructField(oldbalanceOrg,DoubleType,true),StructField(newbalanceOrig,DoubleType,true),StructField(nameDest,StringType,true),StructField(oldbalanceDest,DoubleType,true),StructField(newbalanceDest,DoubleType,true),StructField(isFraud,IntegerType,true),StructField(isFlaggedFraud,IntegerType,true)))

In [49]:
#Register the DataFrame as a SQL temporary view

df.createOrReplaceTempView("finance")

In [50]:
#Test counting how many rows is in the SQL View

Test = spark.sql("SELECT COUNT(*) AS NumberOfRows FROM finance")
Test.show()

+------------+
|NumberOfRows|
+------------+
|     6362620|
+------------+



# Data Mining Process

In [51]:
# 1. Jumlah transaksi per tipe yang terjadi

Q1 = spark.sql("SELECT type as TransactionType, COUNT(type) AS Total FROM finance GROUP BY type")
Q1.show()

+---------------+-------+
|TransactionType|  Total|
+---------------+-------+
|       TRANSFER| 532909|
|        CASH_IN|1399284|
|       CASH_OUT|2237500|
|        PAYMENT|2151495|
|          DEBIT|  41432|
+---------------+-------+



In [52]:
# 2. Mencari Transaksi dengan Jumlah uang dalam transaksi yang paling banyak

Q2 = spark.sql("SELECT * FROM finance WHERE amount = (SELECT MAX(amount) FROM finance)")
Q2.show()

+----+--------+-------------+-----------+-------------+--------------+----------+--------------+--------------+-------+--------------+
|step|    type|       amount|   nameOrig|oldbalanceOrg|newbalanceOrig|  nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+-------------+-----------+-------------+--------------+----------+--------------+--------------+-------+--------------+
| 276|TRANSFER|9.244551664E7|C1715283297|          0.0|           0.0|C439737079|       9595.98| 9.245511262E7|      0|             0|
+----+--------+-------------+-----------+-------------+--------------+----------+--------------+--------------+-------+--------------+



In [53]:
# 3. Jumlah Fraud yg terjadi

Q3 = spark.sql("SELECT SUM(isFraud) AS TotalFraud FROM finance")
Q3.show()

+----------+
|TotalFraud|
+----------+
|      8213|
+----------+



In [54]:
# 4. Jumlah transaksi yang ditandai sebagai Fraud

Q4 = spark.sql("SELECT SUM(isFlaggedFraud) AS TotalFlaggedASFraud FROM finance")
Q4.show()

+-------------------+
|TotalFlaggedASFraud|
+-------------------+
|                 16|
+-------------------+



In [55]:
# 5. Jumlah Transaksi yang memindahkan uang lebih dari 50.000

Q5 = spark.sql("SELECT Count(amount) AS Total FROM finance WHERE oldbalanceOrg-newbalanceOrig >= 50000")
Q5.show()

+------+
| Total|
+------+
|477565|
+------+



In [56]:
# 6. List Customer yang melakukan transaksi pada hari ke 7

Q6 = spark.sql("SELECT DISTINCT nameOrig AS Customer FROM finance WHERE step <=168 and step > 145")
Q6.show()

+-----------+
|   Customer|
+-----------+
|C1460694498|
|C1990967983|
| C299941121|
|C1729943422|
|C1962597060|
| C218035452|
| C234620038|
| C257048083|
|C1470845070|
|C1509275105|
|C1999414572|
| C962302306|
| C785784728|
|C1574306546|
| C232430838|
| C957364590|
|C1022049431|
|C1113810805|
|C1363461862|
| C514110980|
+-----------+
only showing top 20 rows



In [57]:
# 7. Mencari Tanggal berapa yang memiliki jumlah transaksi paling banyak

Q7 = spark.sql("SELECT (step%24+1) AS TransactionDate, COUNT(step) AS Total FROM finance GROUP BY TransactionDate \
                ORDER BY Total DESC LIMIT 1")
Q7.show()

+---------------+------+
|TransactionDate| Total|
+---------------+------+
|             20|647814|
+---------------+------+



In [58]:
# 8. List Recipient yang mengalami penurunan saldo

Q8 = spark.sql("SELECT DISTINCT nameDest AS Recipient FROM finance WHERE newbalanceDest-oldbalanceDest < 0")
Q8.show()

+-----------+
|  Recipient|
+-----------+
|  C39713115|
|C1530856786|
|C1788050799|
| C497806226|
|C1039865382|
|C1827559894|
| C522632546|
| C294252921|
| C445613834|
|  C50060430|
|C1411451832|
| C717575644|
| C486388931|
|C1417414644|
|C1752591265|
|C1796712971|
| C880331985|
|  C62375945|
|C1517552713|
|  C14137534|
+-----------+
only showing top 20 rows



In [59]:
# 9. List Customer yang melakukan transaksi dari tanggal 15 hingga 21 (inclusive)

Q9 = spark.sql("SELECT DISTINCT nameOrig AS Customer FROM finance WHERE step >= 360 and step <= 504")
Q9.show()

+-----------+
|   Customer|
+-----------+
| C548036299|
| C693994331|
| C848252215|
|  C42223398|
|C1272101499|
| C494513285|
| C938847804|
| C464178878|
|C1147221050|
|C2119459805|
| C607933763|
|  C21308903|
|C1872848880|
|C1747555160|
|C1805414807|
|C1027158865|
| C424793600|
| C247349616|
|C1016478517|
| C248386059|
+-----------+
only showing top 20 rows



In [60]:
# 10. List Customer yang memiliki balance lebih dari 100.000 setelah transaksi

Q10 = spark.sql("SELECT DISTINCT nameOrig AS Customer, newbalanceOrig AS Balance FROM finance WHERE newBalanceOrig > 100000")
Q10.show()

+-----------+----------+
|   Customer|   Balance|
+-----------+----------+
|C1614133563| 371688.15|
|C1921143808|2394376.85|
| C168398198| 9822207.6|
|C1561936660|5347521.57|
|C1056094437|3478977.61|
| C586681910|  926816.0|
|C1805660016| 253151.34|
| C111105514|3776336.45|
| C197080602| 292689.28|
|  C20910666|5837237.45|
| C724022349|7206813.28|
|C1795542379| 794086.49|
| C612811551| 157336.89|
|C1480495645|2722218.28|
|C1583425429|1133342.03|
|C1227605734| 497797.47|
|C1389988979| 136547.07|
|C1854532358| 802129.31|
|C2095763698|4070680.13|
|C1591980638|8216737.38|
+-----------+----------+
only showing top 20 rows

